## Inverted Index from Stackoverflow data

### Step 1: Collect data from Stackoverflow using SQL
#### We used the following command to get data from the site http://data.stackexchange.com/stackoverflow/query/new :
*Select* id,  Tags

*from* posts

*where* CreationDate BETWEEN '01/01/2013' *AND* '12/31/2013'

*AND* AnswerCount >1 *AND* Tags <> ''

*ORDER BY* NEWID()

### Step 2: Load findspark so Spark Context is available in code and load pyspark to use Python

In [ ]:
import findspark
findspark.init('/usr/hdp/2.6.3.0-235/spark2')

import pyspark
sc = pyspark.SparkContext(appName="SOMapping")

### Step 3: Move BaseData.csv file into local vagrant file and then to HDFS and read it into Jupyter Notebook

In [59]:
# can load as csv or txt file
text_file2=sc.textFile("hdfs:///user/vagrant/BaseData.csv") # testing which loads better - .txt or .csv  

text_file=sc.textFile("hdfs:///user/vagrant/BaseData.txt", minPartitions=3) #.txt  #3 VM's?

# check that file is an RDD
from pyspark.rdd import RDD
isinstance(text_file, RDD)

text_file2.take(5)

['id,Tags',
 '"19448711","<javascript><javascript-events><cross-browser><onkeyup>"',
 '"16042613","<java>"',
 '"17933344","<python><postgresql><fetchall>"',
 '"15837853","<exception><activemq><jmx>"']

# THIS IS NOT WORKING - filter out first row

In [60]:
header = text_file.first() #extract header
text_file = text_file.filter(row => row != header)   #filter out header

SyntaxError: invalid syntax (<ipython-input-60-985a94428c23>, line 2)

### Step 4: Map the file by replacing '<' and '>', converting to lowercase, and spliting the tags

In [61]:
text_file_comma = text_file.map(lambda x: x.lower().replace('<', ' ').replace('>', '').replace('"', '').split()) #replaces <, >
text_file_comma.take(5) 
# output is a list of lists (of strings)  

[['id', 'tags'],
 ['19448711', 'javascript', 'javascript-events', 'cross-browser', 'onkeyup'],
 ['16042613', 'java'],
 ['17933344', 'python', 'postgresql', 'fetchall'],
 ['15837853', 'exception', 'activemq', 'jmx']]

### Step 5: Use flatMap to get each tag with related Post ID on a unique line

In [62]:
test = text_file_comma.flatMap(lambda list: [(element, list[0]) for element in list[1:]] )
test.take(20)

[('tags', 'id'),
 ('javascript', '19448711'),
 ('javascript-events', '19448711'),
 ('cross-browser', '19448711'),
 ('onkeyup', '19448711'),
 ('java', '16042613'),
 ('python', '17933344'),
 ('postgresql', '17933344'),
 ('fetchall', '17933344'),
 ('exception', '15837853'),
 ('activemq', '15837853'),
 ('jmx', '15837853'),
 ('java', '14944882'),
 ('c++', '20601118'),
 ('php', '16505917'),
 ('c++', '18993197'),
 ('arrays', '18993197'),
 ('pointers', '18993197'),
 ('memory', '18993197'),
 ('dynamic', '18993197')]

### Step 6: Reduce the file to make a list of each tag with the associated post ids

In [63]:
test2 =test.reduceByKey(lambda a,b: a + ',' + b)
test2.take(5)

[('tags',
  'id,20077858,20817230,17472061,19913064,14731380,16330901,19593785,20194271,14829726,18678603,14374265,15917677,14413526,14366182,19769173,19937442,20069658,20464028,17767136,14244220,16968479,16757687,16790722,17426451,19769469,16372299,17632799,17735715,18970704,18250404'),
 ('javascript',
  '19448711,17539449,16182963,19888637,19902017,15454992,19596551,17123706,15382064,17340373,19607874,19079241,18286925,14634138,17533806,18370542,20635469,17731425,20173927,16923385,15634575,16428895,20337869,14187680,20089865,16292349,15235933,18393040,14172343,20139300,17399897,15451959,14314867,16428413,20309746,19401511,14357824,17798278,16282045,17185225,15418134,19398346,16739908,16264634,18763046,16822781,18703906,15387939,19426323,17400191,15774020,20353834,16160622,14655605,19654195,20583551,18919849,20219004,18518812,18060564,19274386,20306204,18016899,17567583,17962353,17460116,19690108,17021873,17663358,14699856,20476255,14133559,15815663,14671204,20243208,18054461,16995108

### Step 7: Add key-value pairs to a dictionary

In [56]:
test3 = test2.collectAsMap()  #Return the key-value pairs in this RDD to the master as a dictionary.
test4 = []  # create python dictionary
for tag in test3:  # for each key value pair in test3, add to dictionary and split on comma
    test4.extend((tag, test3[tag].split(',')))
test4[:5] #return first 5 key,value pairs

['tags',
 ['id',
  '20077858',
  '20817230',
  '17472061',
  '19913064',
  '14731380',
  '16330901',
  '19593785',
  '20194271',
  '14829726',
  '18678603',
  '14374265',
  '15917677',
  '14413526',
  '14366182',
  '19769173',
  '19937442',
  '20069658',
  '20464028',
  '17767136',
  '14244220',
  '16968479',
  '16757687',
  '16790722',
  '17426451',
  '19769469',
  '16372299',
  '17632799',
  '17735715',
  '18970704',
  '18250404'],
 'javascript',
 ['19448711',
  '17539449',
  '16182963',
  '19888637',
  '19902017',
  '15454992',
  '19596551',
  '17123706',
  '15382064',
  '17340373',
  '19607874',
  '19079241',
  '18286925',
  '14634138',
  '17533806',
  '18370542',
  '20635469',
  '17731425',
  '20173927',
  '16923385',
  '15634575',
  '16428895',
  '20337869',
  '14187680',
  '20089865',
  '16292349',
  '15235933',
  '18393040',
  '14172343',
  '20139300',
  '17399897',
  '15451959',
  '14314867',
  '16428413',
  '20309746',
  '19401511',
  '14357824',
  '17798278',
  '16282045',
 

### Step 8: Testing the Inverted Index


# References:
1. http://data.stackexchange.com/stackoverflow/query/new - Getting Stack Overflow Data
2. https://blog.sicara.com/get-started-pyspark-jupyter-guide-tutorial-ae2fe84f594f - starting pyspark Jupyter notebook
3. http://spark.apache.org/docs/2.1.0/api/python/pyspark.html  - Pyspark documentation
4. https://stackoverflow.com/questions/27854919/how-do-i-skip-a-header-from-csv-files-in-spark - Skipping Headers
5. https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_Cheat_Sheet_Python.pdf - Cheat Sheet